In [ ]:
# Import the dependencies.
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [ ]:
# Import citipy module.
from citipy import citipy

In [ ]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

In [ ]:
# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [ ]:
# Starting URL for Weather Map API Call.
url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&lat=35&lon=139&appid=" + weather_api_key

In [ ]:
# Import the time library 
import time

In [ ]:
# Create an empty list to hold the weather data.
city_data = []

# Create counters
record_count = 1
set_count = 1

In [ ]:
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

In [ ]:
# Loop through all the cities in our list.
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)
    
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    #Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        weather_description = city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Current Description": weather_description,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          })

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [15]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Chapais,CA,49.7834,-74.8492,35.38,70,100,11.77,overcast clouds
1,Ponta Delgada,PT,37.7333,-25.6667,63.32,82,20,12.66,few clouds
2,Hithadhoo,MV,-0.6000,73.0833,83.17,67,78,6.29,broken clouds
3,Chuy,UY,-33.6971,-53.4616,66.43,58,77,8.12,broken clouds
4,Port Lincoln,AU,-34.7333,135.8667,65.32,85,100,12.01,overcast clouds
5,The Valley,AI,18.2170,-63.0578,81.82,69,40,20.71,scattered clouds
6,Bengkulu,ID,-3.8004,102.2655,76.93,90,100,3.91,overcast clouds
7,Saint George,US,37.1041,-113.5841,79.63,14,0,35.68,haze
8,Natal,BR,-5.7950,-35.2094,86.65,70,40,13.80,scattered clouds
9,Bluff,NZ,-46.6000,168.3333,38.80,80,96,7.78,overcast clouds


In [ ]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")